In [1]:
import pandas as pd
import numpy as np
import re
from completing_missing_clinician_questionnaire_2021 import map_additional_column_name_to_2021_clinician_column_name, should_remove_from_stu_columns, map_additional_column_name_to_2021_student_column_name , fill_id, IsClinician, upload_to_imputation_data_to_old_data_map, upload_to_old_data_column_names_map
import psycopg2
conn_str = "dbname=postgres user=postgres password=asdf123"


In [2]:
conn = psycopg2.connect(conn_str)
sql = "SELECT * FROM auxiliary_questionnaires_data.questionnaires_columns_names;"
df = pd.read_sql_query(sql, conn)
conn.close()

In [3]:
clinician_questionnaires = ['c_ssrs_clin', 'mini_kid_sum_clin', 'screening_form', 'suicide_form_clin', 'ffq', 'cdrsr_clin', 'scs_clin', 'er_questionnaire_clin', 'cps_clin', 'remote_clin', 'opening_therapist_battery', 'cgi_s_clin', 'trq_sf_maris_clin', 'maris_y_scars_clin', 'wai_immirisk_clin']

In [4]:
clinician_start = 'opening_clinicians_complete'
clinician_end_2021_2022 = 'cssrs_t_clin_complete'#, 'cssrs_fw_maya_complete'
clinician_end_add = 'cssrs_t_complete'


maya_start_2022 = 'cssrs_fw_maya_timestamp'
maya_end_2022 = 'cssrs_fw_maya_complete'



In [5]:
old_data = pd.read_csv(r"../../Data/OriginalDataset/Schneider Depression Clinic Database.csv", na_values= ' ')
redcap_data = pd.read_csv(r"../../Data/OriginalDataset/ImmiRiskIPT2022_DATA_2023-09-03_1503.csv", na_values= ' ')
imputation_data = pd.read_csv(r"../../Data/helper_docs/Student_Clinician_data_2021.csv", na_values= ' ')
# the warning rooted from the nan values


C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,46,51,53,99,325,366,439,453,459,464,466,501,507,512,514,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1271,1281,1283,1285,1291,1293,1297,1305,1306,1319,1321,1324,1326,1338,1340,1342,1343,1344,1350,1351,1352,1353,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types.Speci

In [6]:


rename_imputation_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
}

rename_old_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
    'chameleon_complete_stu': 'chameleon_complete'
}

## error record_id = 363
imputation_data = imputation_data[imputation_data['record_id']!=363]

#do the imputation id like of redcap dataset
imputation_data = fill_id(imputation_data)


imputation_data = imputation_data.rename(rename_imputation_data, axis=1)

old_data = old_data.rename(rename_old_data, axis=1)



In [7]:
old_data_column_names = list(old_data.columns)
redcap_data_column_names = list(redcap_data.columns)
imputation_data_column_names = list(imputation_data.columns)


# clin column: in old dataset and not in redcap 
c_ssrs_t_2weeks_1_clin

In [10]:


### clinician columns
old_data_clinician_columns = old_data_column_names[old_data_column_names.index(clinician_start):old_data_column_names.index(clinician_end_2021_2022) + 1]


redcap_data_clinician_columns = redcap_data_column_names[redcap_data_column_names.index(clinician_start):redcap_data_column_names.index(clinician_end_2021_2022) + 1]

redcap_data_clinician_columns += redcap_data_column_names[redcap_data_column_names.index(maya_start_2022):redcap_data_column_names.index(maya_end_2022) + 1]

imputation_data_clinician_columns = imputation_data_column_names[imputation_data_column_names.index(clinician_start):imputation_data_column_names.index(clinician_end_add)+1]


# imputation -> old data

In [9]:
imputation_2_old_map = {
    # imputation_data_clinician_columns : redcap_data_clinician_columns
    'trqsfmaris_timestamp': 'trqsfmarisclin_timestamp',
    'trqsfmaris_complete': 'trqsfmarisclin_complete'
}


for clinician_column in imputation_data_clinician_columns:
    new_name, success = map_additional_column_name_to_2021_clinician_column_name(clinician_column, old_data_clinician_columns)
    if success:
        imputation_2_old_map[clinician_column] = new_name

     

trqsfmaris_timestamp
trqsfmaris_complete


# Old data -> redcap data

In [11]:
old_2_redcap_map = {
        # redcap_data_clinician_columns: 
}

for clinician_column in old_data_clinician_columns:
    new_name, success = map_additional_column_name_to_2021_clinician_column_name(clinician_column, redcap_data_clinician_columns)
    if success:
        old_2_redcap_map[clinician_column] = new_name
#     else:
#         map_for_clinician_data[clinician_column] = None


c_ssrs_t_2weeks_1_clin


In [11]:
len(imputation_2_old_map.keys())

336

In [12]:
len(old_data_clinician_columns)

336

In [13]:
len(imputation_data_clinician_columns)

336

In [17]:
for questionnaire in clinician_questionnaires:
    
    redcap_columns = df.loc[df['questionnaire_name'] == questionnaire, 'column_names'].values[0]
    questionnaire_old_2_redcap_map = {key: old_2_redcap_map[key] for key in redcap_columns if key in old_2_redcap_map}
    old_data_columns = list(questionnaire_old_2_redcap_map.values())
    questionnaire_imputation_2_old_map = {key: imputation_2_old_map[key] for key in old_data_columns if key in imputation_2_old_map}
    
    
    print(questionnaire, '\n')
    print(old_data_columns, '\n\n')
    
    print('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\')
    
    
#     upload_to_old_data_column_names_map(conn_str, questionnaire, questionnaire_old_2_redcap_map)
    
#     upload_to_imputation_data_to_old_data_map(conn_str, questionnaire, questionnaire_imputation_2_old_map)

c_ssrs_clin 

['cssrs_t_clin_timestamp', 'c_ssrs_t_time_clin', 'c_ssrs_t_2weeks_2_clin', 'c_ssrs_t_2weeks_3_clin', 'c_ssrs_t_2weeks_4_clin', 'c_ssrs_t_2weeks_5_clin', 'c_ssrs_t_last_1_clin', 'c_ssrs_t_last_2_clin', 'c_ssrs_t_last_3_clin', 'c_ssrs_t_last_4_clin', 'c_ssrs_t_last_5_clin', 'c_ssrs_t_life_1_clin', 'c_ssrs_t_life_2_clin', 'c_ssrs_t_life_3_clin', 'c_ssrs_t_life_4_clin', 'c_ssrs_t_life_5_clin', 'c_ssrs_t_6_clin', 'c_ssrs_t_7_clin', 'c_ssrs_t_8_clin', 'c_ssrs_t_9_clin', 'c_ssrs_t_10_clin', 'c_ssrs_t_11_2weeks_clin', 'c_ssrs_t_12_2weeks_clin', 'c_ssrs_t_13_2weeks_clin', 'c_ssrs_t_14_2weeks_clin', 'c_ssrs_t_15_2weeks_clin', 'c_ssrs_t_16_2weeks_clin', 'suicidal_behavior_last_11_clin', 'suicidal_behavior_last_12_clin', 'suicidal_behavior_last_13_clin', 'suicidal_behavior_last_14_clin', 'suicidal_behavior_last_15_clin', 'suicidal_behavior_last_16_clin', 'c_ssrs_t_11_life_clin', 'c_ssrs_t_12_life_clin', 'c_ssrs_t_13_life_clin', 'c_ssrs_t_14_life_clin', 'c_ssrs_t_15_life_clin', 'c_ssr